In [3]:
import os
import merlin.models.torch as mm
import pytorch_lightning as pl
from merlin.dataloader.torch import Loader
from merlin.datasets.entertainment import get_movielens

input_path = os.environ.get("INPUT_DATA_DIR", os.path.expanduser("~/merlin-models-data/movielens/"))
train, valid = get_movielens(variant="ml-1m", path=input_path)
train.schema

,name,tags,dtype,is_list,is_ragged,properties.start_index,properties.cat_path,properties.max_size,properties.num_buckets,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.freq_threshold,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,userId,"(Tags.CATEGORICAL, Tags.USER, Tags.ID)","DType(name='int32', element_type=<ElementType....",False,False,0.0,.//categories/unique.userId.parquet,0.0,NaN,210.0,6041.0,0.0,0.0,6041.0,userId,NaN,NaN
1,movieId,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ID)","DType(name='int32', element_type=<ElementType....",False,False,0.0,.//categories/unique.movieId.parquet,0.0,NaN,159.0,3685.0,0.0,0.0,3685.0,movieId,NaN,NaN
2,title,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,.//categories/unique.title.parquet,0.0,NaN,159.0,3685.0,0.0,0.0,3685.0,title,NaN,NaN
3,genres,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",True,True,0.0,.//categories/unique.genres.parquet,0.0,NaN,16.0,19.0,0.0,0.0,19.0,genres,1.0,6.0
4,gender,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,.//categories/unique.gender.parquet,0.0,NaN,16.0,3.0,0.0,0.0,3.0,gender,NaN,NaN
5,age,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,.//categories/unique.age.parquet,0.0,NaN,16.0,8.0,0.0,0.0,8.0,age,NaN,NaN
6,occupation,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,.//categories/unique.occupation.parquet,0.0,NaN,16.0,22.0,0.0,0.0,22.0,occupation,NaN,NaN
7,zipcode,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,.//categories/unique.zipcode.parquet,0.0,NaN,153.0,3440.0,0.0,0.0,3440.0,zipcode,NaN,NaN
8,TE_age_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float32', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TE_gender_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float32', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
schema = train.schema.without(['rating'])
model = mm.DLRMModel(
    schema, 
    dim=100, 
    bottom_block=mm.MLPBlock([256, 100]),
    top_block=mm.MLPBlock([100, 1]),
    output_block=mm.BinaryOutput(schema['rating_binary'])
)

/home/marc/anaconda3/envs/torchrec-pip/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
trainer = pl.Trainer(max_epochs=1, devices=1)

with Loader(train, batch_size=2048) as train_loader:
    model.initialize(train_loader)
    trainer.fit(model, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/marc/anaconda3/envs/torchrec-pip/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/do

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [5]:
with Loader(valid, batch_size=2048) as valid_loader:
    trainer.validate(model, valid_loader)

MisconfigurationException: No `validation_step()` method defined to run `Trainer.validate`.